# Section 2.3: Probability Theory and Stochastic Processes - Exercises

This notebook contains practical exercises to reinforce your understanding of probability theory and stochastic processes as applied to complex systems modeling.

## Learning Objectives
- Apply probability concepts to analyze random phenomena in complex systems
- Implement and visualize stochastic processes using Python
- Develop intuition for how randomness affects system dynamics
- Practice Bayesian analysis for uncertainty quantification

Let's start by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set plotting style
plt.style.use('ggplot')
sns.set_context("notebook", font_scale=1.2)

# Set random seed for reproducibility
np.random.seed(42)

## Exercise 1: Markov Chain Modeling of Population Movement (Solved)

### Problem Statement

A city is divided into three districts: Downtown (D), Suburbs (S), and Rural Area (R). Each year, residents may move between districts according to the following transition probabilities:

- People in Downtown (D):
  - Stay in Downtown: 70%
  - Move to Suburbs: 25% 
  - Move to Rural Area: 5%
  
- People in Suburbs (S):
  - Move to Downtown: 15%
  - Stay in Suburbs: 75%
  - Move to Rural Area: 10%
  
- People in Rural Area (R):
  - Move to Downtown: 5%
  - Move to Suburbs: 20%
  - Stay in Rural Area: 75%

The current population distribution is: Downtown (40%), Suburbs (35%), and Rural Area (25%).

**Tasks:**
1. Create the transition matrix for this Markov chain
2. Calculate the population distribution after 1, 5, 10, and 20 years
3. Determine if the system reaches a steady state (stationary distribution)
4. Visualize how the population distribution changes over time

### Solution

Let's approach this step by step.

In [ ]:
# Step 1: Create the transition matrix
# Rows represent current state (D, S, R) and columns represent next state (D, S, R)
transition_matrix = np.array([
    [0.70, 0.25, 0.05],  # From Downtown: to D, to S, to R
    [0.15, 0.75, 0.10],  # From Suburbs: to D, to S, to R
    [0.05, 0.20, 0.75]   # From Rural: to D, to S, to R
])

# Display the transition matrix
districts = ['Downtown', 'Suburbs', 'Rural']
transition_df = pd.DataFrame(transition_matrix, 
                             columns=[f'To {d}' for d in districts],
                             index=[f'From {d}' for d in districts])
print("Transition Matrix:")
transition_df

Next, let's define the initial population distribution and calculate future distributions.

In [ ]:
# Step 2: Calculate future distributions
# Initial distribution: Downtown (40%), Suburbs (35%), Rural Area (25%)
initial_distribution = np.array([0.40, 0.35, 0.25])

# Calculate distributions for years 1, 5, 10, and 20
years_to_check = [1, 5, 10, 20]
future_distributions = {}

# Year 0 (initial)
future_distributions[0] = initial_distribution

# Calculate each future year distribution
for year in range(1, max(years_to_check) + 1):
    # For year n, multiply the distribution at year n-1 by the transition matrix
    future_distributions[year] = future_distributions[year-1] @ transition_matrix
    
# Display the results for the specified years
result_data = []
for year in [0] + years_to_check:  # Include initial state (year 0)
    dist = future_distributions[year]
    result_data.append([year] + dist.tolist())
    
result_df = pd.DataFrame(result_data, columns=['Year', 'Downtown', 'Suburbs', 'Rural'])
result_df = result_df.set_index('Year')
result_df

Now let's check if the system reaches a steady state by calculating the distribution over a longer time period.

In [ ]:
# Step 3: Check for steady state / stationary distribution
# Calculate for 100 years
steady_state_years = 100
dist = initial_distribution.copy()

# Store all distributions for visualization
all_distributions = [dist.copy()]

for year in range(1, steady_state_years + 1):
    dist = dist @ transition_matrix
    all_distributions.append(dist.copy())
    
# Convert to DataFrame for easier analysis
all_dists_df = pd.DataFrame(all_distributions, columns=districts)
all_dists_df['Year'] = range(steady_state_years + 1)
all_dists_df = all_dists_df.set_index('Year')

# Calculate differences between consecutive years to check convergence
max_changes = []
for year in range(1, steady_state_years):
    diff = np.abs(all_dists_df.iloc[year] - all_dists_df.iloc[year-1])
    max_changes.append(diff.max())
    
# Find when the change becomes very small (e.g., < 0.0001)
convergence_threshold = 0.0001
steady_state_year = next((i for i, change in enumerate(max_changes) if change < convergence_threshold), None)

if steady_state_year is not None:
    print(f"System reaches approximate steady state after {steady_state_year+1} years")
    print(f"Steady state distribution: {all_dists_df.iloc[steady_state_year+1].values}")
else:
    print("System did not reach steady state within 100 years")
    
# We can also calculate the theoretical steady state directly
# For a regular Markov chain, we can find the eigenvector with eigenvalue 1
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)
# Find the index of eigenvalue closest to 1
one_eigval_idx = np.argmin(np.abs(eigenvalues - 1))
# Get the corresponding eigenvector
stationary_dist = np.real(eigenvectors[:, one_eigval_idx] / eigenvectors[:, one_eigval_idx].sum())

print(f"\nTheoretical steady state: {stationary_dist}")

Finally, let's visualize how the population distribution changes over time.

In [ ]:
# Step 4: Visualize population distribution changes
plt.figure(figsize=(12, 6))

# Plot the first 50 years for better visibility
years_to_plot = 50
plot_data = all_dists_df.iloc[:years_to_plot+1]

for district in districts:
    plt.plot(plot_data.index, plot_data[district], marker='o', markersize=4, label=district)

# Add horizontal lines for steady state values
for i, district in enumerate(districts):
    plt.axhline(y=stationary_dist[i], color=f'C{i}', linestyle='--', alpha=0.7)

plt.xlabel('Year')
plt.ylabel('Population Proportion')
plt.title('Population Distribution Over Time')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Discussion

From our analysis, we can observe that:

1. The population distribution converges to a steady state relatively quickly.
2. In the long run, the city's population distribution stabilizes regardless of the initial distribution.
3. The theoretical steady state can be calculated directly from the transition matrix.

This steady state represents the equilibrium condition where the flow of people in and out of each district balances out. This is a common property of irreducible and aperiodic Markov chains, where regardless of the starting state, the probability distribution will converge to a unique stationary distribution.

In complex systems modeling, this type of analysis helps us understand the long-term behavior of systems with probabilistic transitions, such as population movements, ecological succession, or market dynamics.

## Exercise 2: Bayesian Analysis for Epidemic Modeling (Your Turn)

### Problem Statement

Epidemiologists are studying the spread of a new disease. Based on limited data, they initially believe that the basic reproduction number (R₀) of the disease is between 1.5 and 3.5, with a most likely value around 2.5. This can be modeled as a prior probability distribution.

After conducting a study, they observe that out of 20 index cases (initially infected individuals), 15 of them infected at least one other person, while 5 did not infect anyone else.

**Tasks:**

1. Model the prior belief about R₀ using an appropriate probability distribution
2. Formulate a likelihood function based on the observed data
3. Calculate the posterior distribution for R₀ using Bayes' theorem
4. Visualize the prior, likelihood, and posterior distributions
5. Calculate a 95% credible interval for R₀ based on the posterior distribution

**Hints:**
- A Beta distribution might be appropriate for modeling the prior belief about R₀ (after appropriate scaling)
- The likelihood can be modeled using a binomial distribution
- You may need to use numerical methods to calculate the posterior
- Remember that Bayes' theorem states: Posterior ∝ Prior × Likelihood

### Your Solution

Complete the code cells below to solve this problem:

In [ ]:
# Step 1: Define the prior distribution for R₀
# Hint: You can use a Beta distribution and scale it to the range [1.5, 3.5]

# TODO: Define parameters for the prior distribution
# Your code here

# Create a grid of R₀ values to evaluate the distributions
r0_values = np.linspace(1.0, 4.0, 1000)

# TODO: Calculate the prior probability density at each R₀ value
# Your code here

# Plot the prior distribution
plt.figure(figsize=(10, 6))
# Your code here
plt.title('Prior Distribution for R₀')
plt.xlabel('R₀ (Basic Reproduction Number)')
plt.ylabel('Probability Density')
plt.grid(True)
plt.show()

In [ ]:
# Step 2: Define the likelihood function
# Given: 15 out of 20 index cases infected at least one other person

# TODO: Calculate the likelihood for each value of R₀
# Hint: You'll need to relate R₀ to the probability of infection
# Your code here

# Plot the likelihood function
plt.figure(figsize=(10, 6))
# Your code here
plt.title('Likelihood Function')
plt.xlabel('R₀ (Basic Reproduction Number)')
plt.ylabel('Likelihood')
plt.grid(True)
plt.show()

In [ ]:
# Step 3: Calculate the posterior distribution using Bayes' theorem
# Posterior ∝ Prior × Likelihood

# TODO: Calculate the unnormalized posterior
# Your code here

# TODO: Normalize the posterior (so it integrates to 1)
# Your code here

# Plot the prior, likelihood, and posterior
plt.figure(figsize=(12, 8))
# Your code here
plt.title('Prior, Likelihood, and Posterior Distributions')
plt.xlabel('R₀ (Basic Reproduction Number)')
plt.ylabel('Probability Density / Scaled Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Step 4: Calculate summary statistics from the posterior

# TODO: Find the maximum a posteriori (MAP) estimate
# Your code here

# TODO: Calculate the mean of the posterior
# Your code here

# TODO: Calculate a 95% credible interval
# Your code here

# Print the results
print(f"MAP estimate for R₀: [Your answer here]")
print(f"Posterior mean for R₀: [Your answer here]")
print(f"95% credible interval for R₀: [Your answer here]")

### Analysis Questions

Once you've completed the exercise, answer these questions:

1. How did the posterior distribution change compared to the prior? Did the data strengthen or weaken our initial beliefs?

2. What would happen to the posterior if we had observed only 10 out of 20 index cases infecting others? What about 20 out of 20?

3. How could you improve this model? What other factors might you consider when modeling disease transmission?

4. In complex systems modeling, why is the Bayesian approach particularly useful compared to frequentist methods?


## Conclusion

These exercises have demonstrated how probability theory and stochastic processes can be applied to model complex systems with uncertainty. In the first exercise, we analyzed a Markov chain model of population movement between urban districts, showing how systems can reach equilibrium states over time. In the second exercise, you practiced Bayesian analysis for epidemiological modeling, updating prior beliefs about disease transmissibility based on observed data.

These probabilistic methods are crucial for complex systems modeling because they allow us to:

1. Incorporate uncertainty and variability into our models
2. Make predictions based on incomplete information
3. Update our beliefs systematically as new data becomes available
4. Understand how random processes can lead to emergent patterns

As you progress through the course, you'll see how these concepts connect with other mathematical frameworks and computational methods for analyzing complex systems.